(notebook exécuté sur Google Colab)

In [ ]:
from google.colab import drive


drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sktime
!pip install tslearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 369.8/369.8 kB 8.5 MB/s eta 0:00:00


# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 10.3.4
- **Date :** 16/11/2023
- **Heure :** 20:30
- **Données :** `model_weatherAUS.csv`
- **Tri :** index chronologique
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** non
- **Rééquilibrage :** non
- **Conversion :** `to_sktime_dataset`
- **Algorithme :** ROCKET

## Sommaire

1. Initialisation
2. Découpage
3. Conversion
4. Modélisation
5. Évaluation
6. Enregistrement

## 1. Initialisation

In [ ]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [ ]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/projet/model_weatherAUS.csv", index_col = 1).sort_index()
df = df.drop(columns = "Unnamed: 0")

In [ ]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,Month,Day,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum,NonMesNum
Date,,,,,,,,,,,,,,,,,,,,,
2007-11-01,8.0,24.3,0.0,3.4,6.3,30.0,6.0,20.0,68.0,29.0,...,11,1,15,2.356194,3.926991,2.356194,-35.297591,149.101268,1,2
2007-11-02,14.0,26.9,3.6,4.4,9.7,39.0,4.0,17.0,80.0,36.0,...,11,2,15,0.392699,0.000000,3.141593,-35.297591,149.101268,1,2
2007-11-03,13.7,23.4,3.6,5.8,3.3,85.0,6.0,6.0,82.0,69.0,...,11,3,15,2.356194,1.570796,1.178097,-35.297591,149.101268,1,2
2007-11-04,13.3,15.5,39.8,7.2,9.1,54.0,30.0,24.0,62.0,56.0,...,11,4,15,2.356194,2.748894,3.141593,-35.297591,149.101268,1,2
2007-11-05,7.6,16.1,2.8,5.6,10.6,50.0,20.0,28.0,68.0,49.0,...,11,5,15,5.105088,5.105088,5.890486,-35.297591,149.101268,1,2


In [ ]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140787 entries, 2007-11-01 to 2017-06-25
Data columns (total 29 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   MinTemp         140787 non-null  float64
 1   MaxTemp         140787 non-null  float64
 2   Rainfall        140787 non-null  float64
 3   Evaporation     140787 non-null  float64
 4   Sunshine        140787 non-null  float64
 5   WindGustSpeed   140787 non-null  float64
 6   WindSpeed9am    140787 non-null  float64
 7   WindSpeed3pm    140787 non-null  float64
 8   Humidity9am     140787 non-null  float64
 9   Humidity3pm     140787 non-null  float64
 10  Pressure9am     140787 non-null  float64
 11  Pressure3pm     140787 non-null  float64
 12  Cloud9am        140787 non-null  float64
 13  Cloud3pm        140787 non-null  float64
 14  Temp9am         140787 non-null  float64
 15  Temp3pm         140787 non-null  float64
 16  RainToday       140787 non-null  int64  
 17  Ra

## 2. Découpage

In [ ]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [ ]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

In [ ]:
# Vérification de la longueur de `X_train`

len(X_train)

117323

## 3. Conversion

In [ ]:
# Conversion des jeux `X_train` et `X_test` au format `sktime` avec avec la fonction `to_sktime_dataset` de la bibliothèque `tslearn`

from tslearn.utils import to_sktime_dataset

X_train = to_sktime_dataset(X_train)
X_test = to_sktime_dataset(X_test)

## 4. Modélisation

In [ ]:
# Instanciation d'un transformateur ROCKET

from sktime.transformations.panel.rocket import Rocket

rocket = Rocket()

In [ ]:
# Récupération des paramètres initiaux du transformateur ROCKET

rocket.get_params()

{'n_jobs': 1, 'normalise': True, 'num_kernels': 10000, 'random_state': None}

In [ ]:
# Ajustement du transformateur ROCKET au jeu d'entraînement

rocket.fit(X_train)

Rocket()

In [ ]:
# Récupération des paramètres ajustés du transformateur ROCKET

rocket.get_fitted_params()

{}

In [ ]:
# Application du transformateur ROCKET au jeu d'entraînement

X_train_transform = rocket.transform(X_train)

In [ ]:
# Instanciation d'un modèle classificateur linéaire

from sklearn.linear_model import SGDClassifier

clf_rocket_sgd_log = SGDClassifier(loss = "log_loss") ## Pour `len(X_train) > 20000`, `sktime` recommande d'utiliser la régression logistique entraînée à l'aide de la descente de gradient stochastique (_stochastic gradient descent_ / SGD).

In [ ]:
# Récupération des paramètres initiaux du modèle

clf_rocket_sgd_log.get_params()

{'alpha': 0.0001,
 'average': False,
 'class_weight': None,
 'early_stopping': False,
 'epsilon': 0.1,
 'eta0': 0.0,
 'fit_intercept': True,
 'l1_ratio': 0.15,
 'learning_rate': 'optimal',
 'loss': 'log_loss',
 'max_iter': 1000,
 'n_iter_no_change': 5,
 'n_jobs': None,
 'penalty': 'l2',
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'tol': 0.001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [ ]:
# Entraînement du modèle

clf_rocket_sgd_log.fit(X_train_transform, y_train)

SGDClassifier(loss='log_loss')

In [ ]:
# Récupération des paramètres ajustés du modèle

clf_rocket_sgd_log.get_params()

{'alpha': 0.0001,
 'average': False,
 'class_weight': None,
 'early_stopping': False,
 'epsilon': 0.1,
 'eta0': 0.0,
 'fit_intercept': True,
 'l1_ratio': 0.15,
 'learning_rate': 'optimal',
 'loss': 'log_loss',
 'max_iter': 1000,
 'n_iter_no_change': 5,
 'n_jobs': None,
 'penalty': 'l2',
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'tol': 0.001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [ ]:
# Application du transformateur ROCKET au jeu de test

X_test_transform = rocket.transform(X_test)

In [ ]:
# Réalisation des prédictions

y_pred = clf_rocket_sgd_log.predict(X_test_transform)

## 5. Évaluation

In [ ]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0,1
Classe réelle,,
0,18052,67
1,4548,797


In [ ]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89     18119
           1       0.92      0.15      0.26      5345

    accuracy                           0.80     23464
   macro avg       0.86      0.57      0.57     23464
weighted avg       0.83      0.80      0.74     23464



## 6. Enregistrement

In [ ]:
from joblib import dump

dump(clf_rocket_sgd_log, "/content/drive/MyDrive/Colab Notebooks/projet/10.3.4-clf-rocket-sgd-log.joblib")

['/content/drive/MyDrive/Colab Notebooks/projet/10.3.4-clf-rocket-sgd-log.joblib']